# [Lv4-Day4-Lab4] The Grand Unification: An A2A Integrated Research Blog System

### 실습 목표
지금까지 우리가 구축한 모든 개별 프레임워크와 프로토콜을 A2A(Agent-to-Agent)과 유사한 시스템을 직접 구현해봅니다. 우리는 외부 세계에 단 하나의 **통합 API 엔드포인트**를 제공하고, 이 엔드포인트 뒤에서 `LangGraph`, `CrewAI`, `ADK` Agent들이 유기적으로 협력하여 복잡한 블로그 생성 작업을 완수하는 최종 아키텍처를 구현합니다.

이번 **Part 1**에서는 이 거대한 시스템의 '언어'와 '관문'을 정의하는, 즉 **A2A 통신 프로토콜**과 **FastAPI 게이트웨이**의 뼈대를 구축하는 데 집중합니다.

1.  **A2A 프로토콜 설계:** 각 전문 Agent 시스템(대화 관리, 콘텐츠 생성, 품질 관리)이 서로 주고받을 데이터의 형식을 `Pydantic` 모델을 사용하여 명확한 **'API 명세(Contract)'**로 정의합니다.
2.  **FastAPI 게이트웨이 구축:** 모든 외부 요청을 받아들이는 단일 진입점(Single Point of Entry) 역할을 할 `FastAPI` 서버를 구축합니다. 이 게이트웨이는 요청을 분석하여 내부의 적절한 전문 Agent 시스템으로 전달하는 라우팅 역할을 합니다.
3.  **서비스 스텁(Stub) 구현:** 실제 Agent 로직을 연동하기 전에, 각 엔드포인트가 올바르게 작동하는지 확인할 수 있도록 간단한 '가짜(Mock)' 응답을 반환하는 스텁(Stub)을 구현하여 시스템의 기본 구조를 검증합니다.

### 0. 사전 준비: 라이브러리 설치 및 환경 설정
이 실습은 4일차의 모든 기술 스택을 총망라합니다. `FastAPI`를 포함한 모든 관련 라이브러리를 설치하고, 서버 보안을 위한 `MASTER_API_KEY`를 설정합니다.

In [ ]:
# !pip install fastapi uvicorn python-dotenv nest-asyncio aiohttp httpx pydantic langchain-google-genai crewai crewai_tools langgraph requests -q

In [ ]:
import os
from getpass import getpass

# 기본 API 키들 설정
os.environ["GOOGLE_API_KEY"] = getpass("GOOGLE_API_KEY 를 입력하세요: ")
os.environ["TAVILY_API_KEY"] = getpass("TAVILY_API_KEY 를 입력하세요: ")
os.environ["NEWS_API_KEY"] = getpass("NEWS_API_KEY 를 입력하세요: ")
os.environ["MASTER_API_KEY"] = "samsung-llm-agent-lv4-master-key"
print("✅ 모든 API 키가 성공적으로 설정되었습니다.")

### 1. A2A 프로토콜: 시스템의 '공용 언어' 정의

서로 다른 프레임워크로 만들어진 Agent들이 원활하게 소통하기 위해서는, 모두가 이해할 수 있는 **'공용 언어'**, 즉 표준화된 데이터 형식이 필요합니다. 우리는 `Pydantic`을 사용하여 각 핵심 기능(대화 관리, 콘텐츠 생성, 품질 관리)에 대한 요청(Request)과 응답(Response)의 데이터 구조를 명확하게 정의합니다. 이것이 바로 우리 시스템의 A2A(Agent-to-Agent) 프로토콜입니다.

In [ ]:
%%writefile a2a_protocol.py

from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any

# --- 1. 대화 관리 (LangGraph Agent 담당) ---
class DialogueRequest(BaseModel):
    session_id: str
    user_input: str

class DialogueResponse(BaseModel):
    session_id: str
    agent_response: str
    next_action: Optional[Dict[str, Any]] = None # 예: {'action': 'CREATE_CONTENT', 'topic': '...'

# --- 2. 콘텐츠 생성 (CrewAI Team 담당) ---
class ContentCreationRequest(BaseModel):
    topic: str
    user_preferences: str

class ContentCreationResponse(BaseModel):
    draft_content: str
    status: str # 'COMPLETED', 'FAILED'
    error_message: Optional[str] = None

# --- 3. 품질 관리 (ADK Agent 담당) ---
class QualityControlRequest(BaseModel):
    topic: str
    draft_content: str

class QualityControlResponse(BaseModel):
    final_post: str
    qa_report: Dict[str, Any] # {'seo_score': 85, 'readability': 'good', ...}
    status: str

print("✅ A2A 통신을 위한 Pydantic 프로토콜 모델이 정의되었습니다.")

### 2. A2A 게이트웨이: 시스템의 '단일 창구' 구현

이제 A2A 프로토콜을 사용하는 **`FastAPI` 게이트웨이**를 구현합니다. 이 서버는 우리 시스템의 유일한 '공식 창구' 역할을 하며, 외부 세계와 내부의 복잡한 Agent 시스템들 사이를 중계합니다.

이번 파트에서는 각 엔드포인트에 실제 Agent 로직을 연동하기 전에, **간단한 '스텁(Stub)' 로직**을 넣어 서버의 기본 구조가 올바르게 작동하는지 먼저 검증합니다. 또한, Lab 1에서 구현했던 **API 키 인증** 보안 계층을 처음부터 적용하여, 허가된 요청만이 우리 시스템에 접근할 수 있도록 합니다.

In [ ]:
%%writefile a2a_blog_system.py

from fastapi import FastAPI, Depends, HTTPException, Security
from fastapi.security import APIKeyHeader
import os
from a2a_protocol import (
    DialogueRequest, DialogueResponse,
    ContentCreationRequest, ContentCreationResponse,
    QualityControlRequest, QualityControlResponse
)

# --- 1. FastAPI 앱 및 보안 설정 ---
app = FastAPI(
    title="A2A Integrated Research Blog System",
    version="1.0",
)

API_KEY_NAME = "X-API-Key"
api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=True)

async def get_api_key(api_key: str = Security(api_key_header)):
    if api_key == os.environ.get("MASTER_API_KEY"):
        return api_key
    else:
        raise HTTPException(status_code=401, detail="Invalid API Key")

# --- 2. 통합 API 엔드포인트 (스텁 버전) ---

@app.post("/dialogue", response_model=DialogueResponse, tags=["A2A Protocol"])
async def handle_dialogue(request: DialogueRequest, api_key: str = Depends(get_api_key)):
    """사용자와의 대화를 관리하고, 다음 행동을 결정합니다. (LangGraph Agent 담당)"""
    print(f"[Gateway] /dialogue 요청 수신: {request.user_input}")
    # (TODO: Part 2에서 실제 LangGraph 로직 연동)
    return DialogueResponse(
        session_id=request.session_id,
        agent_response=f"'{request.user_input}'에 대한 응답입니다. 콘텐츠 생성을 시작합니다.",
        next_action={"action": "CREATE_CONTENT", "topic": request.user_input}
    )

@app.post("/create-content", response_model=ContentCreationResponse, tags=["A2A Protocol"])
async def handle_content_creation(request: ContentCreationRequest, api_key: str = Depends(get_api_key)):
    """주어진 주제로 블로그 초안을 생성합니다. (CrewAI Team 담당)"""
    print(f"[Gateway] /create-content 요청 수신: {request.topic}")
    # (TODO: Part 2에서 실제 CrewAI 로직 연동)
    return ContentCreationResponse(
        draft_content=f"## {request.topic}에 대한 블로그 초안\n\n이것은 CrewAI 팀이 작성한 초안입니다...",
        status="COMPLETED"
    )

@app.post("/quality-control", response_model=QualityControlResponse, tags=["A2A Protocol"])
async def handle_quality_control(request: QualityControlRequest, api_key: str = Depends(get_api_key)):
    """생성된 초안의 품질을 검증하고 최종본을 만듭니다. (ADK Agent 담당)"""
    print(f"[Gateway] /quality-control 요청 수신: {request.topic}")
    # (TODO: Part 2에서 실제 ADK 로직 연동)
    return QualityControlResponse(
        final_post=f"## {request.topic} (최종 편집본)\n\n{request.draft_content}",
        qa_report={"seo_score": 95, "readability": "excellent"},
        status="COMPLETED"
    )

print("✅ A2A 게이트웨이 서버(a2a_blog_system.py)가 생성되었습니다.")

### 3. 서버 실행 유틸리티: FastAPI 앱 런처 구현

Lab 1에서 구현했던 **환경 감지형 서버 실행기**를 A2A 게이트웨이용으로 재구현합니다.

**구현 특징:**
- **포트 충돌 방지**: 기존 실행 중인 서버 프로세스를 안전하게 정리

- **IP 인증 지원**: External URL 접근 시 필요한 IP 주소 자동 확인 및 안내

이 함수는 이후 우리의 A2A 게이트웨이 서버(`a2a_blog_system.py`)를 실행할 때 사용됩니다.

In [ ]:
import sys
import subprocess
import time
from IPython.display import display, HTML


def launch_fastapi_local(app_filename="a2a_blog_system.py", port=8502):
    """
    FastAPI 앱(Uvicorn)을 백그라운드에서 실행하고 로컬 접속 URL을 생성합니다.
    (local_url, server_process) 튜플을 반환합니다.
    """
    # Step 1: 기존 프로세스 정리
    try:
        subprocess.run(["pkill", "-f", "uvicorn"], capture_output=True)
        print("기존 FastAPI(Uvicorn) 프로세스를 정리했습니다.")
        time.sleep(2)
    except Exception as e:
        pass

    # Step 2: FastAPI 앱(Uvicorn) 백그라운드 실행
    try:
        app_name = app_filename.replace(".py", "")
        command = [sys.executable, "-m", "uvicorn", f"{app_name}:app", "--host", "0.0.0.0", "--port", str(port)]
        server_process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"FastAPI 서버를 포트 {port}에서 시작하는 중...")
        time.sleep(5)  # 서버가 완전히 시작될 때까지 잠시 대기
    except Exception as e:
        print(f"FastAPI 서버 시작에 실패했습니다: {e}")
        return None, None

    # Step 3: 결과 출력
    local_url = f"http://localhost:{port}"
    print("\n" + "=" * 60)
    print("A2A Gateway 서버가 성공적으로 시작되었습니다!")
    print("=" * 60)
    print(f"다음 URL을 클릭하여 API 서버에 접속하세요:")
    print(f"🔗 {local_url}")

    try:
        display(HTML(f'<b><a href="{local_url}" target="_blank">👉 여기를 클릭하여 서버에 접속하세요</a></b>'))
    except NameError:
        pass

    return local_url, server_process

### 4. 전문가 서비스 모듈화: 각 Agent 시스템을 독립된 '부품'으로 만들기

우리의 통합 서버(`a2a_blog_system.py`)가 비대해지고 복잡해지는 것을 막기 위해, 각 전문 Agent 시스템의 실행 로직을 별도의 Python 파일로 분리합니다. 이는 각 팀이 자신의 코드에만 집중할 수 있게 하고, 시스템 전체의 구조를 명확하게 만드는 중요한 설계 패턴입니다.

**구현 내용:**
1.  **`dialogue_manager_langgraph.py`:** 사용자와의 초기 상호작용을 담당하고, 다음 행동(콘텐츠 생성 요청 등)을 결정하는 `LangGraph` 기반의 대화 관리자 로직을 구현합니다.
2.  **`content_creation_crew.py`:** `CrewAI`의 계층적 팀을 사용하여 실제 블로그 초안을 생성하는 로직을 구현합니다. 여기서 핵심은, 이 팀의 `Research Agent`가 더 이상 로컬 Tool을 사용하는 것이 아니라, **Lab 1에서 만든 `MCPClient`를 통해 중앙 서버의 Tool을 호출**하도록 하여, 우리의 분산 아키텍처에 완벽하게 편입시키는 것입니다.
3.  **`quality_control_adk.py`:** `ADK Agent`의 개념을 차용하여, 생성된 초안의 품질을 검증하고 최종 편집하는 로직을 구현합니다.

In [ ]:
%%writefile dialogue_manager_langgraph.py

import os
from langchain_google_genai import ChatGoogleGenerativeAI

async def handle_dialogue_logic(user_input: str):
    """LangGraph Agent의 역할을 시뮬레이션하는 대화 관리 로직"""
    print("[LangGraph Service] 🧠 대화 관리자 실행됨...")

    try:
        # Google API 키 확인
        if not os.environ.get("GOOGLE_API_KEY"):
            print("[LangGraph Service] ⚠️ Google API 키가 설정되지 않음. Mock 응답 사용.")
            topic = f"AI 기술 주제: {user_input[:50]}"
        else:
            llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)
            prompt = f"다음 사용자 요청의 핵심 주제를 한 문장으로 요약해줘: '{user_input}'"
            topic = llm.invoke(prompt).content

        response = {
            "agent_response": f"알겠습니다. '{topic}'에 대한 블로그 글 생성을 시작하겠습니다.",
            "next_action": {
                "action": "CREATE_CONTENT",
                "topic": topic,
                "user_preferences": "전문적이면서도 쉬운 어조로 작성해주세요."
            }
        }
        print(f"[LangGraph Service] ✅ 다음 행동 결정: {response['next_action']}")
        return response

    except Exception as e:
        print(f"[LangGraph Service] ❌ 오류 발생: {e}")
        # 폴백 응답
        return {
            "agent_response": f"'{user_input}'에 대한 블로그 글 생성을 시작하겠습니다.",
            "next_action": {
                "action": "CREATE_CONTENT",
                "topic": user_input,
                "user_preferences": "전문적이면서도 쉬운 어조로 작성해주세요."
            }
        }

print("✅ dialogue_manager_langgraph.py 파일이 생성되었습니다.")

In [ ]:
%%writefile content_creation_crew.py

import os
import json
import requests
from langchain_google_genai import ChatGoogleGenerativeAI

# 간단한 MCPClient 구현
class MCPClient:
    def __init__(self, base_url: str, api_key: str):
        self.base_url = base_url
        self.headers = {"X-API-Key": api_key, "Content-Type": "application/json"}

    def call_tool(self, tool_name: str, query: str) -> dict:
        endpoint = f"{self.base_url}/api/v1/tools/{tool_name}"
        payload = {"query": query}
        try:
            response = requests.post(endpoint, headers=self.headers, json=payload, timeout=30)
            if response.status_code == 200:
                return response.json()
            else:
                return {"error": f"HTTP {response.status_code}", "detail": response.text}
        except Exception as e:
            return {"error": f"요청 실패: {e}"}

async def handle_creation_logic(topic: str, user_preferences: str, mcp_client=None):
    """CrewAI 팀을 시뮬레이션하는 콘텐츠 생성 로직"""
    print("[CrewAI Service] 👥 콘텐츠 제작팀 가동됨...")

    try:
        # Google API 키 확인
        if not os.environ.get("GOOGLE_API_KEY"):
            print("[CrewAI Service] ⚠️ Google API 키가 설정되지 않음. Mock 응답 사용.")
            return f"""# {topic}

## 개요
{topic}에 대한 포괄적인 분석입니다.

## 주요 내용
1. 기술 개요
2. 작동 원리
3. 미래 전망

## 결론
{topic}는 미래 기술 발전의 핵심 요소입니다.

*CrewAI 팀이 생성한 초안입니다.*
"""

        # 1. Research 단계 (MCP 서버 활용)
        research_content = ""
        if mcp_client:
            print("[CrewAI Service] 🔍 MCP 서버를 통한 리서치 수행...")
            search_result = mcp_client.call_tool("web_search", topic)
            if "error" not in search_result:
                research_content = f"리서치 결과: {search_result.get('result', '')[:200]}..."

        # 2. Writing 단계
        print("[CrewAI Service] ✍️ 블로그 초안 작성...")
        llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

        prompt = f"""
당신은 전문 기술 블로거입니다. 다음 주제로 블로그 글을 작성해주세요:

주제: {topic}
사용자 선호도: {user_preferences}
리서치 자료: {research_content}

구조:
1. 매력적인 제목
2. 서론 (독자 관심 유발)
3. 본론 (핵심 내용, 구체적 예시)
4. 결론 (요약 및 미래 전망)

전문적이면서도 이해하기 쉽게 작성해주세요.
"""

        result = llm.invoke(prompt).content
        print("[CrewAI Service] ✅ 초안 작성 완료.")
        return result

    except Exception as e:
        print(f"[CrewAI Service] ❌ 오류 발생: {e}")
        # 폴백 응답
        return f"""# {topic}

## 기술 개요
{topic}에 대한 상세한 분석을 제공합니다.

## 핵심 내용
- 최신 기술 동향
- 실제 활용 사례
- 미래 발전 방향

## 결론
{topic}는 앞으로 더욱 중요한 기술이 될 것입니다.

*시스템에서 자동 생성된 초안입니다.*
"""

print("✅ content_creation_crew.py 파일이 생성되었습니다.")

In [ ]:
%%writefile quality_control_adk.py

from langchain_google_genai import ChatGoogleGenerativeAI

async def handle_qc_logic(topic: str, draft_content: str):
    """ADK Agent의 역할을 시뮬레이션하는 품질 관리 로직"""
    print("[ADK Service] 🧐 품질 관리팀 가동됨...")
    qa_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    prompt = f"당신은 수석 편집자입니다. 다음 블로그 초안을 최종 검토하고, 발행 가능한 완벽한 최종본으로 만들어주세요.\n주제: {topic}\n초안: {draft_content}"
    final_post = qa_llm.invoke(prompt).content

    report = {"seo_score": 95, "readability": "excellent", "final_char_count": len(final_post)}
    print("[ADK Service] ✅ 최종 편집 및 보고서 생성 완료.")
    return final_post, report

print("✅ quality_control_adk.py 파일이 생성되었습니다.")

### 5. 최종 통합: 게이트웨이 서버에 실제 서비스 연동

이제 마지막으로, 우리의 메인 서버 파일인 `a2a_blog_system.py`를 수정하여, 스텁(Stub) 로직을 방금 만든 실제 서비스 모듈들을 호출하는 코드로 교체합니다. 또한, 비동기 `FastAPI` 환경에서 동기적으로 작동하는 `CrewAI`와 같은 라이브러리를 안전하게 실행하기 위해 `asyncio.to_thread`를 사용하는 실무적인 기술을 적용합니다.

In [ ]:
%%writefile a2a_blog_system.py

from fastapi import FastAPI, Depends, HTTPException, Security
from fastapi.security import APIKeyHeader
import os
import asyncio
from typing import Optional
from a2a_protocol import *
from dialogue_manager_langgraph import handle_dialogue_logic
from content_creation_crew import handle_creation_logic, MCPClient
from quality_control_adk import handle_qc_logic

# FastAPI 앱 및 보안 설정
app = FastAPI(
    title="A2A Integrated Research Blog System",
    description="다양한 Agent 프레임워크를 통합한 블로그 생성 시스템",
    version="1.0.0"
)

API_KEY_NAME = "X-API-Key"
api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=True)

async def get_api_key(api_key: str = Security(api_key_header)):
    if api_key == os.environ.get("MASTER_API_KEY"):
        return api_key
    else:
        raise HTTPException(status_code=401, detail="Invalid API Key")

# MCP 클라이언트 전역 인스턴스
mcp_client: Optional[MCPClient] = None

@app.on_event("startup")
def startup_event():
    global mcp_client
    mcp_server_url = os.environ.get("MCP_SERVER_URL")
    if mcp_server_url:
        mcp_client = MCPClient(
            base_url=mcp_server_url,
            api_key=os.environ.get("MASTER_API_KEY")
        )
        print(f"✅ MCP 클라이언트 초기화 완료: {mcp_server_url}")
    else:
        print("⚠️ MCP_SERVER_URL이 설정되지 않음")

# 통합 API 엔드포인트
@app.get("/", summary="서버 상태 확인")
async def read_root():
    return {"status": "ok", "message": "A2A Gateway Server is running"}

@app.post("/dialogue", response_model=DialogueResponse, tags=["A2A Protocol"])
async def handle_dialogue(request: DialogueRequest, api_key: str = Depends(get_api_key)):
    """사용자와의 대화를 관리하고, 다음 행동을 결정합니다. (LangGraph Agent 담당)"""
    print(f"[Gateway] /dialogue 요청 수신: {request.user_input}")
    try:
        result = await handle_dialogue_logic(request.user_input)
        return DialogueResponse(session_id=request.session_id, **result)
    except Exception as e:
        print(f"[Gateway] Dialogue 오류: {e}")
        return DialogueResponse(
            session_id=request.session_id,
            agent_response=f"죄송합니다. 요청 처리 중 오류가 발생했습니다: {str(e)}",
            next_action=None
        )

@app.post("/create-content", response_model=ContentCreationResponse, tags=["A2A Protocol"])
async def handle_content_creation(request: ContentCreationRequest, api_key: str = Depends(get_api_key)):
    """주어진 주제로 블로그 초안을 생성합니다. (CrewAI Team 담당)"""
    print(f"[Gateway] /create-content 요청 수신: {request.topic}")
    try:
        # 비동기 환경에서 안전하게 실행
        draft = await handle_creation_logic(request.topic, request.user_preferences, mcp_client)
        return ContentCreationResponse(draft_content=draft, status="COMPLETED")
    except Exception as e:
        print(f"[Gateway] Content Creation 오류: {e}")
        return ContentCreationResponse(
            draft_content="",
            status="FAILED",
            error_message=str(e)
        )

@app.post("/quality-control", response_model=QualityControlResponse, tags=["A2A Protocol"])
async def handle_quality_control(request: QualityControlRequest, api_key: str = Depends(get_api_key)):
    """생성된 초안의 품질을 검증하고 최종본을 만듭니다. (ADK Agent 담당)"""
    print(f"[Gateway] /quality-control 요청 수신: {request.topic}")
    try:
        final_post, report = await handle_qc_logic(request.topic, request.draft_content)
        return QualityControlResponse(
            final_post=final_post,
            qa_report=report,
            status="COMPLETED"
        )
    except Exception as e:
        print(f"[Gateway] Quality Control 오류: {e}")
        return QualityControlResponse(
            final_post=request.draft_content,  # 폴백으로 원본 반환
            qa_report={"error": str(e)},
            status="FAILED"
        )

print("✅ 모든 전문 서비스가 통합된 최종 a2a_blog_system.py가 생성되었습니다.")

### 6. 최종 통합 서버 재실행

Part 2에서 우리는 `a2a_blog_system.py` 파일과 각 전문가 서비스 모듈(`dialogue_manager_langgraph.py` 등)을 모두 업데이트했습니다. 이제 이 최신 버전의 코드가 반영된 통합 서버를 다시 시작하겠습니다.

**⚠️ 매우 중요:**
이 통합 서버(`a2a_blog_system.py`)는 내부적으로 Lab 1에서 만든 **MCP 서버(`mcp_server.py`)**를 호출합니다. 따라서 이 실습을 진행하기 전에, **반드시 Lab 1의 `ipynb` 파일로 돌아가 MCP 서버를 먼저 실행**하고, 그 Public URL을 아래 코드의 `mcp_server_url` 변수에 입력해야 합니다. 두 개의 서버가 동시에 실행되어야 이 시스템이 완벽하게 작동합니다.

In [ ]:
import os

# 이전 실습에서 8501 포트로 실행한 MCP 서버의 로컬 주소를 사용합니다.
mcp_server_url = "http://localhost:8501"
os.environ["MCP_SERVER_URL"] = mcp_server_url

# A2A Gateway 서버 실행
if mcp_server_url:
    print(f"MCP 서버 URL이 '{mcp_server_url}'로 설정되었습니다.")

    # 이전 셀에서 수정한 launch_fastapi_local 함수를 호출합니다
    a2a_server_url, a2a_server_process = launch_fastapi_local("a2a_blog_system.py", 8502)

    if a2a_server_url:
        print(f"\n최종 통합 서버(A2A Gateway)가 성공적으로 시작되었습니다.")
        print(f"   Local URL: {a2a_server_url}")
        print(f"   API 문서: {a2a_server_url}/docs")
    else:
        print("A2A Gateway 서버 시작에 실패했습니다.")
else:
    print("MCP 서버 URL을 먼저 설정해야 합니다.")

### 7. 최종 오케스트레이션 시뮬레이션: '지휘자' 클라이언트 실행

이제 모든 것이 준비되었습니다. 우리는 '최종 사용자' 또는 '상위 애플리케이션'의 입장에서, 우리가 만든 단일 통합 엔드포인트를 순차적으로 호출하여 블로그 생성 프로젝트 전체를 지휘하는 클라이언트 로직을 실행합니다.

이 클라이언트는 다음과 같은 역할을 수행합니다:
1.  **`/dialogue` 호출:** 사용자의 초기 입력을 대화 관리자(`LangGraph`)에게 보내, 작업의 핵심 주제를 확정하고 다음 행동 지시를 받습니다.
2.  **`/create-content` 호출:** 대화 관리자의 지시에 따라, 확정된 주제를 콘텐츠 제작팀(`CrewAI`)에게 전달하여 초안 생성을 요청합니다.
3.  **`/quality-control` 호출:** 생성된 초안을 품질 관리팀(`ADK`)에게 전달하여 최종 편집 및 검수를 요청합니다.
4.  **최종 결과 출력:** 최종적으로 완성된 블로그 글을 사용자에게 보여줍니다.

In [ ]:
import requests
import time
import json
import uuid

if "a2a_public_url" in locals() and a2a_public_url:
    # 서버가 완전히 시작될 때까지 잠시 대기
    time.sleep(10)

    # --- 1. 시뮬레이션 준비 ---
    session_id = f"session_{uuid.uuid4()}"
    initial_user_input = "최신 AI 기술인 'LLM Agent'의 작동 원리와 미래 전망에 대해 기술 블로그 글을 써줘."
    headers = {"X-API-Key": os.environ["MASTER_API_KEY"]}

    print("--- 🚀 최종 오케스트레이션 시뮬레이션 시작 ---")
    print(f"사용자 요청: {initial_user_input}")

    try:
        # --- 2. [API Call 1] 대화 관리자 호출 (LangGraph) ---
        print("\n1️⃣  LangGraph 대화 관리자에게 작업 분석 요청...")
        dialogue_req = {"session_id": session_id, "user_input": initial_user_input}
        dialogue_res = requests.post(f"{a2a_public_url}/dialogue", json=dialogue_req, headers=headers)
        dialogue_res.raise_for_status()
        dialogue_data = dialogue_res.json()
        print(f"   - 응답: {dialogue_data['agent_response']}")
        print(f"   - 다음 행동: {dialogue_data['next_action']}")

        # --- 3. [API Call 2] 콘텐츠 제작팀 호출 (CrewAI) ---
        if dialogue_data.get("next_action", {}).get("action") == "CREATE_CONTENT":
            print("\n2️⃣  CrewAI 콘텐츠 제작팀에게 초안 생성 요청...")
            creation_req = dialogue_data["next_action"]
            creation_res = requests.post(f"{a2a_public_url}/create-content", json=creation_req, headers=headers)
            creation_res.raise_for_status()
            creation_data = creation_res.json()
            print(f"   - 상태: {creation_data['status']}")
            draft_content = creation_data["draft_content"]
            print(f"   - 생성된 초안 (일부): {draft_content[:100]}...")

            # --- 4. [API Call 3] 품질 관리팀 호출 (ADK) ---
            print("\n3️⃣  ADK 품질 관리팀에게 최종 편집 요청...")
            qc_req = {"topic": creation_req["topic"], "draft_content": draft_content}
            qc_res = requests.post(f"{a2a_public_url}/quality-control", json=qc_req, headers=headers)
            qc_res.raise_for_status()
            qc_data = qc_res.json()
            print(f"   - 상태: {qc_data['status']}")
            final_post = qc_data["final_post"]

            # --- 5. 최종 결과물 출력 ---
            print("\n" + "=" * 80)
            print("                           ✅ 최종 블로그 포스트 ✅")
            print("=" * 80)
            print(final_post)
        else:
            print("\n- 대화 관리자가 콘텐츠 생성 액션을 반환하지 않았습니다.")

    except requests.exceptions.RequestException as e:
        print(f"\n❌ API 호출 중 오류 발생: {e}")
        if e.response is not None:
            print(f"   - 응답 내용: {e.response.text}")

else:
    print("❌ A2A Gateway 서버 URL이 설정되지 않았습니다. 이전 셀을 다시 실행해주세요.")